# collections of results

My [StackOverflow question](https://stackoverflow.com/questions/70881768/avoiding-nesting-resultt-terror) about handling collections of `Result<'T,'TError>` recognizes the problem of getting “too many” F# results.

Suppose we have this `input`:

In [ ]:
#!fsharp

let input =
    [
        "0.10"
        "0.21"
        "forty"
        "5.32"
        "q6.20"
    ]

Mapping this input to F# results via `tryParseDecimal` leads to `Result<'T,'TError> list`:

In [ ]:
#!fsharp

open System

let tryParseDecimal (input: string) =
    match Decimal.TryParse input with
    | true, i -> Ok i
    | _ -> Error $"failed to parse `{input}`"

let output = input |> List.map tryParseDecimal

output |> printf "%A"

[Ok 0.10M; Ok 0.21M; Error "failed to parse `forty`"; Ok 5.32M;
 Error "failed to parse `q6.20`"]

Depending on the intent of the architect, this could be “too many” instances of `Result<'T,'TError>` as the error handling sub-system is built for _one_ instance of `Result<'T,'TError>` (which is classic, relatively straight-forward _railway-oriented programming_ [📖 [docs](https://swlaschin.gitbooks.io/fsharpforfunandprofit/content/posts/recipe-part2.html)]).

## enter `FsToolkit.ErrorHandling`

One module-based way to address collections of F# results is with `FsToolkit.ErrorHandling` [[GitHub](https://github.com/demystifyfp/FsToolkit.ErrorHandling)]:

In [ ]:
#r "nuget: FsToolkit.ErrorHandling"

Installed Packages FsToolkit.ErrorHandling, 2.13.0

`FsToolkit.ErrorHandling` is “opinionated” about handling collections of results by providing these relevant functions:

| name | remarks | function signature |
|- |- |-
| `List.sequenceResultA` | returns an `Ok` list _or_ an `Error` list | `Result<'a, 'b> list -> Result<'a list, 'b list>` |
| `List.sequenceResultM` | returns an `Ok` list _or_ the first `Error` | `Result<'a, 'b> list -> Result<'a list, 'b>` |
| `List.traverseResultA` | returns an `Ok` list _or_ an `Error` list | `('a -> Result<'b,'c>) -> 'a list -> Result<'b list, 'c list>` |
| `List.traverseResultM` | returns an `Ok` list _or_ the first `Error` | `('a -> Result<'b,'c>) -> 'a list -> Result<'b list, 'c>` |

The use of the words `sequence` and `traverse` are deliberate. Scott Wlaschin, his “[Understanding traverse and sequence](https://swlaschin.gitbooks.io/fsharpforfunandprofit/content/posts/elevated-world-4.html),” details the meaning of these words:

>The Traversable world can be stacked on top of the Applicative world, which produces a type such as `List<Result<a>>` …what happens if you are just handed a `List<Result>` and you need to change it to a `Result<List>` …This is where sequence is useful—that’s exactly what it does!

The function signatures in the table above show that `sequence` takes `Result<'a, 'b> list` as input and `traverse` takes a function, `('a -> Result<'b,'c>)`, as input. Both `sequence` and `traverse` produce the same output with different input.

For example, we can take the `output` above and _sequence_ it:

In [ ]:
#!fsharp

open FsToolkit.ErrorHandling

output |> List.sequenceResultA |> printfn "%A"

output |> List.sequenceResultM |> printf "%A"

Error ["failed to parse `forty`"; "failed to parse `q6.20`"]
Error "failed to parse `forty`"

When the architecture provides our `input` directly, we can _traverse_ it:

In [ ]:
#!fsharp

input |> List.traverseResultA tryParseDecimal |> printfn "%A"

input |> List.traverseResultM tryParseDecimal |> printf "%A"

Error ["failed to parse `forty`"; "failed to parse `q6.20`"]
Error "failed to parse `forty`"

[Bryan Wilhite is on LinkedIn](https://www.linkedin.com/in/wilhite)🇺🇸💼
